In [ ]:
import torch
import copy

if torch.cuda.is_available():
    print('Cuda Available')
else:
    print('No GPU support currently')

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import pandas as pd
import numpy as np

# upload the database
file_name = (r'Database.csv') # insert database name here
data = pd.read_csv(file_name)
data.shape

In [ ]:
# Drop rows with missing values in the target column
data = data.dropna(subset=['GEOLOGIAcat'])
# Drop rows with Vs higher than 1500 m/s
data = data[data['VsEquivalente'] <= 1500]

In [ ]:
# Data filtering function based on Vs values
def filtro_vs_intervalli(data):
    
    data = data[((data['GEOLOGIAcat'] == 1) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 375)) |
                (data['GEOLOGIAcat'] != 1)]
    
    
    data = data[((data['GEOLOGIAcat'] == 2) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 650)) |
                (data['GEOLOGIAcat'] != 2)]
    
    
    data = data[((data['GEOLOGIAcat'] == 3) & (data['VsEquivalente'] >= 0) & (data['VsEquivalente'] <= 600)) |
                (data['GEOLOGIAcat'] != 3)]
    
    
    data = data[((data['GEOLOGIAcat'] == 4) & (data['VsEquivalente'] >= 350) & (data['VsEquivalente'] <= 600)) |
                (data['GEOLOGIAcat'] != 4)]
    
    
    data = data[((data['GEOLOGIAcat'] == 5) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 550)) |
                (data['GEOLOGIAcat'] != 5)]
    
    
    data = data[((data['GEOLOGIAcat'] == 6) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 900)) |
                (data['GEOLOGIAcat'] != 6)]
    
    
    data = data[((data['GEOLOGIAcat'] == 7) & (data['VsEquivalente'] >= 0) & (data['VsEquivalente'] <= 500)) |
                (data['GEOLOGIAcat'] != 7)]
    
    
    data = data[((data['GEOLOGIAcat'] == 8) & (data['VsEquivalente'] >= 0) & (data['VsEquivalente'] <= 300)) |
                (data['GEOLOGIAcat'] != 8)]
    
    
    data = data[((data['GEOLOGIAcat'] == 9) & (data['VsEquivalente'] >= 250) & (data['VsEquivalente'] <= 1000)) |
                (data['GEOLOGIAcat'] != 9)]
    
    
    data = data[((data['GEOLOGIAcat'] == 10) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 1000)) |
                (data['GEOLOGIAcat'] != 10)]
    
    
    data = data[((data['GEOLOGIAcat'] == 11) & (data['VsEquivalente'] >= 50) & (data['VsEquivalente'] <= 800)) |
                (data['GEOLOGIAcat'] != 11)]
    
    
    data = data[((data['GEOLOGIAcat'] == 12) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 1500)) |
                (data['GEOLOGIAcat'] != 12)]
    
    
    data = data[((data['GEOLOGIAcat'] == 13) & (data['VsEquivalente'] >= 500) & (data['VsEquivalente'] <= 1200)) |
                (data['GEOLOGIAcat'] != 13)]
    
    
    data = data[((data['GEOLOGIAcat'] == 14) & (data['VsEquivalente'] >= 300) & (data['VsEquivalente'] <= 700)) |
                (data['GEOLOGIAcat'] != 14)]
    
    
    data = data[((data['GEOLOGIAcat'] == 15) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 700)) |
                (data['GEOLOGIAcat'] != 15)]
    
    
    data = data[((data['GEOLOGIAcat'] == 16) & (data['VsEquivalente'] >= 300) & (data['VsEquivalente'] <= 500)) |
                (data['GEOLOGIAcat'] != 16)]
    
    
    data = data[((data['GEOLOGIAcat'] == 17) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 700)) |
                (data['GEOLOGIAcat'] != 17)]
    
    
    data = data[((data['GEOLOGIAcat'] == 18) & (data['VsEquivalente'] >= 200) & (data['VsEquivalente'] <= 900)) |
                (data['GEOLOGIAcat'] != 18)]
    
    
    data = data[((data['GEOLOGIAcat'] == 20) & (data['VsEquivalente'] >= 100) & (data['VsEquivalente'] <= 400)) |
                (data['GEOLOGIAcat'] != 20)]
    
    return data

# Apply the filtering function to the data
data_filtered = filtro_vs_intervalli(data)

In [ ]:
# Filtering the data based on the 'xcoord' and 'ycoord' columns
data_filtered['coord_pair'] = list(zip(data_filtered['xcoord'], data_filtered['xcoord']))

# Drop duplicate rows based on the 'coord_pair' column
data_final = data_filtered.drop_duplicates(subset='coord_pair').reset_index(drop=True)

# Drop the 'coord_pair' column
data_final = data_final.drop(columns='coord_pair')

In [ ]:
# Drop
data_final = data_final.drop(columns=['id', 'spessore_totale', 'tipo_ind'])
data_filtered = data_final.dropna()

In [ ]:
import pandas as pd
from sklearn.utils import resample

# Data augmentation function
def classify_velocity(value):
    if value < 180:
        return '<180'
    elif 180 <= value < 240:
        return '180-240'
    elif 240 <= value < 300:
        return '240-300'
    elif 300 <= value < 360:
        return '300-360'
    elif 360 <= value < 420:
        return '360-420'
    elif 420 <= value < 480:
        return '420-480'
    elif 480 <= value < 560:
        return '480-560'
    elif 560 <= value < 640:
        return '560-640'
    elif 640 <= value < 760:
        return '640-760'
    else:
        return '>760'

# Add a new column 'Class' to the filtered data
data_filtered['Class'] = data_filtered['VsEquivalente'].apply(classify_velocity)

# Count the number of samples in each class
class_counts = data_filtered['Class'].value_counts()

# Define the target class
target_class = '360-420'
target_class_count = class_counts[target_class]

# Find the majority class
majority_class = class_counts.idxmax()  
majority_class_count = class_counts.max()  

# Empty DataFrame to store the balanced data
balanced_data = pd.DataFrame()

# Under-sampling and over-sampling
for class_label, count in class_counts.items():
    class_subset = data_filtered[data_filtered['Class'] == class_label]
    
    if class_label == majority_class:
        
        class_subset_resampled = resample(class_subset,
                                          replace=False,      
                                          n_samples=target_class_count, 
                                          random_state=42)    
    elif count < target_class_count:
        
        class_subset_resampled = resample(class_subset,
                                          replace=True,       
                                          n_samples=target_class_count, 
                                          random_state=42)    
    else:
        
        class_subset_resampled = class_subset
    
    # Add the resampled subset to the balanced data
    balanced_data = pd.concat([balanced_data, class_subset_resampled])

balanced_data = balanced_data.drop(columns=['Class']).reset_index(drop=True)

In [ ]:
# Drop rows with missing values
balanced_data = balanced_data.dropna()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Create a dictionary that maps the lithology code to the lithology name
geologia_mapping = {
    1: ('Ad', '#7F90E0'),
    2: ('CM', '#E87208'),
    3: ('Al', '#A4E1FE'),
    4: ('Gd', '#160BBF'),
    5: ('E', '#F9A906'),
    6: ('Ssr', '#4DC79E'),
    7: ('Mw', '#000000'),
    8: ('Li', '#27CEFB'),
    9: ('Lb', '#FA1301'),
    10: ('M', '#6D2E07'),
    11: ('Pr', '#F47EAB'),
    12: ('Cr', '#EBD3BD'),
    13: ('Ir', '#CC197D'),
    14: ('Nsr', '#EF3DE6'),
    15: ('Sr', '#7130FC'),
    16: ('Ccr', '#FAE802'),
    17: ('Ucr', '#FDFFB7'),
    18: ('SM', '#B9E064'),
    20: ('B', '#FEFF08')
}

# Create a dictionary that maps the lithology code to the lithology color
geologia_colori = {v[0]: v[1] for v in geologia_mapping.values()}

# Create a new column 'GEOLOGIA_sigla' that maps the lithology code to the lithology name
balanced_data['GEOLOGIA_sigla'] = balanced_data['GEOLOGIAcat'].map(lambda x: geologia_mapping[x][0])

# Calculate the average velocity for each lithology
media_velocità = balanced_data.groupby('GEOLOGIA_sigla')['VsEquivalente'].mean().reset_index()

# order the lithologies based on the average velocity
media_velocità = media_velocità.sort_values(by='VsEquivalente', ascending=False)

# Create a list of ordered lithologies
ordered_lithologies = media_velocità['GEOLOGIA_sigla'].tolist()

# Set the figure 
plt.figure(figsize=(12, 8))

sns.boxplot(x='GEOLOGIA_sigla', y='VsEquivalente', data=balanced_data,
            palette=geologia_colori, order=ordered_lithologies)

plt.xlabel('Lithological classes', fontsize=14)
plt.ylabel('Vs,30 [m/s]', fontsize=14)

plt.xticks(rotation=45)
plt.grid(alpha=0.1)

# Save figure 
plt.tight_layout()
#plt.savefig('box_plot_geologie_after_processing.png', dpi=1000)

plt.show()

In [ ]:
# Function to round a number to the nearest multiple of 5
def round_to_nearest_5(x):
    return round(x / 5) * 5

balanced_data['VsEquivalente'] = balanced_data['VsEquivalente'].apply(round_to_nearest_5)

In [ ]:
# Selecting the features and target

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR

# Fixing seeds for reproducibility
torch.manual_seed(5)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)

# convert string to array
def convert_string_to_array(string_array):
    
    if isinstance(string_array, np.ndarray):
        return string_array

    string_values = string_array.strip('[]').split(', ')
    float_array = np.array([float(value) for value in string_values])
    return float_array

# Starting the scaler
scaler = StandardScaler()

# features selection
features = balanced_data[['DEM1', 'SLOPE1', 'ROUGHNESS1', 'RUGGEDNESS1', 'TOPOGRAPHI1', 'PROFILE_CU1', 'TANGENTIAL_CU1', 'ASPECT1', 'DX1', 'DY1', 'GEOLOGIAcat']]
# Target selection
labels = balanced_data[['VsEquivalente']]

In [ ]:
# Normalizing the features
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Upoading the global means and standard deviations for the normalization 
global_means = pd.read_csv('global_means.csv', index_col=0, usecols=[0, 1])
global_stds = pd.read_csv('global_stds.csv', index_col=0, usecols=[0, 1])

# Columns to normalize
columns_to_normalize = ['DEM1', 'SLOPE1', 'ROUGHNESS1', 'RUGGEDNESS1', 
                         'TOPOGRAPHI1', 'PROFILE_CU1', 'TANGENTIAL_CU1', 
                         'ASPECT1', 'DX1', 'DY1']

# columns not to normalize
columns_non_normalized = features.columns.difference(columns_to_normalize)

# Create a StandardScaler object
scaler = StandardScaler()
scaler.mean_ = global_means.values.flatten()
scaler.scale_ = global_stds.values.flatten()

features_selected = features[columns_to_normalize]

features_normalized = scaler.transform(features_selected.values)

# Add the non-normalized columns to the normalized features
features_final = pd.DataFrame(features_normalized, columns=columns_to_normalize)
features_final[columns_non_normalized] = features[columns_non_normalized].values

# 1. Data splitting into training, validation, and test sets
features_train, features_temp, labels_train, labels_temp = train_test_split(
    features_final.values, labels.values, test_size=0.2, random_state=42)

features_val, features_test, labels_val, labels_test = train_test_split(
    features_temp, labels_temp, test_size=0.5, random_state=42)

# 2. Convert the NumPy arrays to PyTorch tensors
features_tensor_train = torch.tensor(features_train, dtype=torch.float32)
labels_tensor_train = torch.tensor(labels_train, dtype=torch.float32)

features_tensor_val = torch.tensor(features_val, dtype=torch.float32)
labels_tensor_val = torch.tensor(labels_val, dtype=torch.float32)

features_tensor_test = torch.tensor(features_test, dtype=torch.float32)
labels_tensor_test = torch.tensor(labels_test, dtype=torch.float32)

# 3. Creation of DataLoaders
train_dataset = TensorDataset(features_tensor_train, labels_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

val_dataset = TensorDataset(features_tensor_val, labels_tensor_val)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

test_dataset = TensorDataset(features_tensor_test, labels_tensor_test)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

In [ ]:
# Model definition and Training

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt
import copy
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error, explained_variance_score

# Defining the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_prob=0.01):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        self.layer3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.layer3(x)

        smooth_range = (x > 1500)
        x = torch.where(smooth_range, 1500 + torch.sigmoid(x - 1500) * (x - 1500), x)
        x = torch.minimum(x, torch.tensor(2000.0).to(x.device))  
        return x


# Model parameters
input_size = 11  # Input features
output_size = 1  # Output features

# Hyperparameters
hidden_sizes = [(4096, 8192) ]
learning_rates = [0.001]
dropout_probs = [0.01]  
l1_lambdas = [0.00001]  
l2_lambdas = [0.00001]  
best_model = None
best_loss = float('inf')
best_configs = []

# Regularized loss function
def loss_with_regularization(outputs, labels, l1_lambda, l2_lambda):
    l1_reg = 0
    l2_reg = 0
    for param in model.parameters():
        l1_reg += torch.norm(param, 1)
        l2_reg += torch.norm(param, 2)
    loss = criterion(outputs, labels)
    loss += l1_lambda * l1_reg
    loss += l2_lambda * l2_reg
    return loss

# Early stopping parameters
patience = 50
patience_counter = 0

plt.figure(figsize=(10, 6))

for (hidden_size1, hidden_size2) in hidden_sizes:
    for lr in learning_rates:
        for dropout_prob in dropout_probs:
            for l1_lambda in l1_lambdas:
                for l2_lambda in l2_lambdas:
                    
                    model = NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size, dropout_prob).to(device)

                    
                    criterion = nn.MSELoss()
                    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=l2_lambda)
                    scheduler = StepLR(optimizer, step_size=100, gamma=0.5)

                    train_losses_config = []
                    val_losses_config = []

                    # Training loop
                    num_epochs = 5000
                    for epoch in range(num_epochs):
                        model.train()
                        train_loss_epoch = 0.0
                        for inputs, labels in train_loader:
                            optimizer.zero_grad()
                            inputs, labels = inputs.to(device), labels.to(device)
                            outputs = model(inputs)
                            loss = loss_with_regularization(outputs, labels, l1_lambda, l2_lambda)
                            loss.backward()
                            optimizer.step()
                            train_loss_epoch += loss.item()
                        scheduler.step()
                        model.eval()

                        train_loss_epoch /= len(train_loader)
                        train_losses_config.append(train_loss_epoch)

                        val_loss_epoch = 0.0
                        num_batches_val = 0
                        all_outputs = []
                        all_labels = []
                        with torch.no_grad():
                            for inputs, labels in val_loader:
                                inputs, labels = inputs.to(device), labels.to(device)
                                outputs = model(inputs)
                                val_loss_epoch += criterion(outputs, labels).item()
                                num_batches_val += 1
                                all_outputs.append(outputs.cpu().numpy())
                                all_labels.append(labels.cpu().numpy())

                        val_loss_epoch /= num_batches_val
                        val_losses_config.append(val_loss_epoch)

                        # Concatenate all outputs and labels for metrics calculation
                        all_outputs = np.concatenate(all_outputs)
                        all_labels = np.concatenate(all_labels)

                        # Print the loss every 10 epochs
                        if (epoch + 1) % 10 == 0:
                            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss_epoch:.4f}, Val Loss: {val_loss_epoch:.4f}, '
                                  f'Config: H1={hidden_size1}, H2={hidden_size2}, LR={lr}, Dropout={dropout_prob}, '
                                  f'L1={l1_lambda}, L2={l2_lambda}')

                        # Early stopping check
                        if val_loss_epoch < best_loss:
                            best_loss = val_loss_epoch
                            best_epoch = epoch
                            best_model = copy.deepcopy(model)  # Save the best model
                            patience_counter = 0
                        else:
                            patience_counter += 1
                            if patience_counter >= patience:
                                print(f"Early stopping at epoch {epoch + 1}")
                                break

                    # Use the best model for evaluation
                    model = best_model  

                    # Statistics calculation
                    mae = mean_absolute_error(all_labels, all_outputs)
                    mse = mean_squared_error(all_labels, all_outputs)
                    rmse = np.sqrt(mse)
                    r2 = r2_score(all_labels, all_outputs)
                    median_ae = median_absolute_error(all_labels, all_outputs)
                    explained_variance = explained_variance_score(all_labels, all_outputs)
                    max_error = np.max(np.abs(all_labels - all_outputs))

                    print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, Median AE: {median_ae:.4f}, Explained Variance: {explained_variance:.4f}, Max Error: {max_error:.4f}')

                    best_configs.append({
                        'hidden_size1': hidden_size1,
                        'hidden_size2': hidden_size2,
                        'learning_rate': lr,
                        'dropout_prob': dropout_prob,
                        'l1_lambda': l1_lambda,
                        'l2_lambda': l2_lambda,
                        'val_loss': val_loss_epoch,
                        'mae': mae,
                        'rmse': rmse,
                        'r2': r2
                    })

                    # plot the loss
                    plt.plot(range(1, len(train_losses_config) + 1), train_losses_config,
                             label=f'Train Loss (H1={hidden_size1}, H2={hidden_size2}, LR={lr})', color='blue')
                    plt.plot(range(1, len(val_losses_config) + 1), val_losses_config,
                             linestyle='--', label=f'Validation Loss (H1={hidden_size1}, H2={hidden_size2}, LR={lr})', color='red')

# add minumum validation loss point 
min_val_loss_epoch = np.argmin(val_losses_config) + 1
min_val_loss = min(val_losses_config)
plt.scatter(min_val_loss_epoch, min_val_loss, color='red', marker='o', s=100,
            label=f'Min Val Loss (Epoch {min_val_loss_epoch})')

# Visualize the plot
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')

plt.legend(loc='upper right')

plt.tight_layout()
plt.grid()

# Save the plot
#plt.savefig('loss_plot.png', dpi=300)
plt.show()

In [ ]:
# Evaluate the model

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            predictions.extend(outputs.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    average_loss = total_loss / len(dataloader)
    rmse = np.sqrt(mean_squared_error(true_labels, predictions))
    r2 = r2_score(true_labels, predictions)

    return average_loss, rmse, r2, predictions, true_labels

# Evaluate the best model on the training, validation, and test sets
train_loss, train_rmse, train_r2, train_predictions, train_true_labels = evaluate_model(best_model, train_loader, nn.MSELoss(), device)
val_loss, val_rmse, val_r2, val_predictions, val_true_labels = evaluate_model(best_model, val_loader, nn.MSELoss(), device)
test_loss, test_rmse, test_r2, test_predictions, test_true_labels = evaluate_model(best_model, test_loader, nn.MSELoss(), device)

print(f'Train Loss: {train_loss:.4f}, Train RMSE: {train_rmse:.4f}, Train R^2 Score: {train_r2:.4f}')
print(f'Validation Loss: {val_loss:.4f}, Validation RMSE: {val_rmse:.4f}, Validation R^2 Score: {val_r2:.4f}')
print(f'Test Loss: {test_loss:.4f}, Test RMSE: {test_rmse:.4f}, Test R^2 Score: {test_r2:.4f}')

# Function to plot the real vs. predicted values
def plot_real_vs_predicted(true_labels, predictions, title):
    plt.figure(figsize=(8, 8))
    plt.scatter(true_labels, predictions, color='blue', alpha=0.7, edgecolor='black')
    plt.plot([min(true_labels), max(true_labels)], [min(true_labels), max(true_labels)], linestyle='--', color='red', linewidth=2)
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title(title)
    plt.grid(True)
    plt.show()

# training set
plot_real_vs_predicted(train_true_labels, train_predictions, 'Real vs. Predicted (Training Set)')

# validation set
plot_real_vs_predicted(val_true_labels, val_predictions, 'Real vs. Predicted (Validation Set)')

# test set
plot_real_vs_predicted(test_true_labels, test_predictions, 'Real vs. Predicted (Test Set)')

In [ ]:
# 1. Salva il modello addestrato in UTM
import json

torch.save(best_model.state_dict(), 'modello_final_100m_30_10_24_HIDDEN2_base_8mila_neuroni_DEM_SLOPE.pth')

# 2. Salva gli iperparametri e altre informazioni utili
model_info = {
    'hidden_size1': hidden_size1,
    'hidden_size2': hidden_size2,
    'learning_rate': lr,
    'best_loss': best_loss,
    'best_epoch': best_epoch,
}
with open('modello_final_100m_30_10_24_HIDDEN2_base_8mila_neuroni_DEM_SLOPE', 'w') as json_file:
    json.dump(model_info, json_file)

# 3. Salva lo stato dell'ottimizzatore
torch.save(optimizer.state_dict(), 'optimizer_state_modello_final_100m_30_10_24_HIDDEN2_base_8mila_neuroni_DEM_SLOPE.pth')

In [ ]:
# Save the model, hyperparameters, and optimizer state

import json
import torch

# 1. Save the trained model
try:
    torch.save(best_model.state_dict(), 'INSERT_NAME_HERE.pth') # insert pth file name here
    print("Model saved.")
except Exception as e:
    print(f"Error during saving: {e}")

# 2. Save the hyperparameters and other useful information
model_info = {
    'hidden_size1': hidden_size1,
    'hidden_size2': hidden_size2,
    'learning_rate': lr,
    'best_loss': best_loss,
    'best_epoch': best_epoch,
}

try:
    with open('INSERT_NAME_HERE.json', 'w') as json_file: # insert json file name here
        json.dump(model_info, json_file)
    print("Model info saved.")
except Exception as e:
    print(f"Error during saving: {e}")

# 3. Save the optimizer state
try:
    torch.save(optimizer.state_dict(), 'INSERT_NAME_HERE.pth') # optimizer
    print("Optimizer saved.")
except Exception as e:
    print(f"Error during saving the optimizer: {e}")